In [1]:
import pandas as pd
import numpy as np
from aeon.datasets import load_electric_devices_segmentation, load_from_tsv_file
from aeon.segmentation import ClaSPSegmenter, find_dominant_window_sizes
from aeon.visualisation import plot_series_with_change_points, plot_series_with_profiles

In [2]:
# read parquet
data = pd.read_parquet('data/parquet_samples/13_06_22/radar_samples_192.168.67.112_412.parquet')
# first six columns are data for segmentation
data_s = data.iloc[:, :6]
col_names = data_s.columns

In [4]:
data_s

In [3]:
# prepare the np array for the data
all_col_l = []
for col in col_names:
    df = data_s[col]
    data_series = pd.DataFrame(df)
    data_series_exploded = data_series.explode(col)
    merged_list = data_series_exploded[col].tolist()
    # normalize
    merged_l_mean = np.mean(merged_list)
    merged_l_std = np.std(merged_list)
    merged_list = (merged_list - merged_l_mean) / merged_l_std

    all_col_l.append(merged_list)

all_col_arr = np.array(all_col_l)
arr_split = np.split(all_col_arr, 10, axis=1)
arr_split

In [14]:
num = 1
for arr in arr_split:
    np.savetxt(f'06_22_radar112_{num}.tsv', all_col_arr, delimiter='\t', fmt='%.8f')
    num += 1

In [8]:
np.savetxt('06_22_radar112.tsv', all_col_arr, delimiter='\t', fmt='%.8f')

In [15]:
X, y = load_from_tsv_file('06_22_radar112_1.tsv')

In [9]:
loaded_array = np.loadtxt('06_22_radar112.tsv', delimiter='\t')
loaded_array

In [6]:
from aeon.datasets import load_electric_devices_segmentation, load_gun_point_segmentation
X, true_period_size, cps = load_gun_point_segmentation()
X

In [ ]:
dominant_period_size = find_dominant_window_sizes(X)
clasp = ClaSPSegmenter(dominant_period_size, n_cps=1)


In [ ]:
found_cps = clasp.fit_predict(X)
profiles = clasp.profiles
scores = clasp.scores

In [13]:
import numpy as np
from igts import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [14]:
data1 = pd.read_parquet('data/parquet_samples/13_06_22/radar_samples_192.168.67.112_412.parquet')
data1 = data1.iloc[:, :6]
data1

In [15]:
col_names = data1.columns
col_names

In [16]:
all_col_l = []
for col in col_names:
    df = data1[col]
    data_series = pd.DataFrame(df)
    data_series_exploded = data_series.explode(col)
    merged_list = data_series_exploded[col].tolist()
    # normalize
    merged_l_mean = np.mean(merged_list)
    merged_l_std = np.std(merged_list)
    merged_list = (merged_list - merged_l_mean) / merged_l_std
    
    all_col_l.append(merged_list)

In [7]:
all_col_arr = np.array(all_col_l)
all_col_arr

In [8]:
data = all_col_arr
# Downsampling factor
downsample_factor = 1000

# Downsample the data by taking every `downsample_factor` point
downsampled_data = data[:, ::downsample_factor]

# Plot each channel as a line plot
plt.figure(figsize=(15, 10))
for i in range(downsampled_data.shape[0]):
    plt.plot(downsampled_data[i], label=f'Channel {i+1}')

plt.title('Downsampled Data Line Plot')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.show()

In [12]:
import matplotlib.pyplot as plt

def plot_segments(data, segments):
    plt.figure(figsize=(15, 6))
    for i in range(data.shape[0]):
        plt.plot(data[i], label=f'Channel {i+1}')
    for seg in segments:
        plt.axvline(x=seg, color='r', linestyle='--')
    plt.title('Time Series Segmentation')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

# Assuming `data` is your time series data and `segments` is the result from TopDown
segments, IG_arr, knee = TopDown(data, k=5, step=100)
plot_segments(data, segments)


In [11]:
# Define the chunk size
chunk_size = 100000

# Number of chunks
num_chunks = data.shape[1] // chunk_size

# Plot each chunk
for chunk in range(num_chunks):
    start = chunk * chunk_size
    end = start + chunk_size
    chunk_data = data[:, start:end]
    
    # TopDown Approach
    Integ_TS = chunk_data
    TD_TT, IG_TT, knee =TopDown(Integ_TS, k=8,1,1)
    segment_points = TD_TT


    
    plt.figure(figsize=(15, 10))
    for i in range(chunk_data.shape[0]):
        plt.plot(chunk_data[i], label=f'Channel {i+1}', alpha=0.3)
        # Overlay segment points
    for point in segment_points:
        plt.axvline(x=point, color='r', linestyle='--', label=f'Segment {point}')

    
    plt.title(f'Data from index {start} to {end}')
    plt.xlabel('Index within chunk')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

In [45]:
Integ_TS = downsampled_data

In [12]:
TS_len=420
TS_dim=6
k=3
Integ_TS = np.zeros((TS_dim, TS_len))
print((TS_len/40)-1)
display(Integ_TS.shape)
display(Integ_TS)

In [4]:
GT_TT = 40 * np.random.randint(1,(TS_len/40)-1, size=k)
GT_TT = GT_TT-1
GT_TT

In [5]:
gt = np.append(np.array(1), np.append(sorted(GT_TT),np.array(TS_len)))
print('GT_TT >>>' , sorted(GT_TT))

In [6]:
Lval=0
for i in range(k):
    for d in range(TS_dim):
        val = np.random.randint(1,6)
        if Lval==val:
            val=val+2
        for j in range(gt[i],gt[i+1],1):
            Integ_TS[d,j] = val
        Lval=val

In [16]:
Integ_TS = Integ_TS /2
mu, sigma = 0, 0.1 
noise = np.random.normal(mu, sigma, [TS_dim,TS_len]) 
Integ_TS = Integ_TS + noise
print(Integ_TS.shape)
display(Integ_TS)

In [46]:
DP_TT,_ =DP_IGTS(Integ_TS, k,1,1)
print('Dynamic Programming extracted TT >>>' , DP_TT)

In [48]:
# TopDown Approach
TD_TT, IG_TT, knee =TopDown(Integ_TS, k,1,1)
print('Top Down Approach:\n extracted  boundaries>>> {}\n Corresponding IG values >>> {} - knee point : {}'.format(TD_TT,IG_TT,knee))

In [49]:
segment_points = TD_TT

In [21]:
# Plot each row as a separate line
plt.figure(figsize=(10, 6))
for i in range(data.shape[0]):
    plt.plot(data[i], label=f'Row {i+1}')

plt.title('Line Plot of the 2D Array')
plt.xlabel('Column Index')
plt.ylabel('Value')
plt.legend()
plt.show()

In [9]:
data = Integ_TS
# Plot each row of data
plt.figure(figsize=(14, 8))
for i in range(data.shape[0]):
    plt.plot(data[i], label=f'Row {i+1}')

# Overlay segment points
for point in segment_points:
    plt.axvline(x=point, color='r', linestyle='--', label=f'Segment {point}')

plt.title('Data with Segment Boundaries')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.show()